In [3]:
!pip install trl
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from trl import RewardTrainer, RewardConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer
from trl import RewardTrainer, RewardConfig
device = "cpu"
# Model ve Tokenizer
configuration = GPT2Config()
GPT2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT2_tokenizer.pad_token = GPT2_tokenizer.eos_token
model = GPT2ForSequenceClassification(configuration).from_pretrained("gpt2").to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model.config.pad_token_id = model.config.eos_token_id

# Add the pad token before tokenizing the data
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer)) # Resize model embeddings to include the new pad token

# Veri Yükleme
dataset = load_dataset("xsum", split="train[:1000]")

# Tokenize edilmiş uygun veri formatı
def preprocess_data(example):
    chosen_text = example["document"]
    rejected_text = example["summary"]

    # Metni tokenize et
    chosen_tokens = tokenizer(chosen_text, truncation=True, max_length=512, padding="max_length", return_tensors="pt")
    rejected_tokens = tokenizer(rejected_text, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

    # Gerekli sütunları oluştur
    return {
        "input_ids_chosen": chosen_tokens["input_ids"].squeeze(0),
        "attention_mask_chosen": chosen_tokens["attention_mask"].squeeze(0),
        "input_ids_rejected": rejected_tokens["input_ids"].squeeze(0),
        "attention_mask_rejected": rejected_tokens["attention_mask"].squeeze(0),
    }

# Dataset üzerinde preprocess işlemi
processed_dataset = dataset.map(preprocess_data, remove_columns=dataset.column_names)

# Training Argümanları
training_args = RewardConfig(
    output_dir="koyun",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=1e-3,
)

# Trainer
trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset,  # Eğitim ve doğrulama için aynı veri kümesi kullanılabilir (örnek için)
)
trainer.train()

In [ ]:
from trl import PPOConfig
config = PPOConfig(output_dir="directoryOutPPO",learning_rate=1.4e-5)

In [ ]:
from trl import AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer

model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
from trl import PPOTrainer
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
ppo_trainer = PPOTrainer(
    config=config,
    train_dataset=dataset,  # Burada uygun bir veri kümesi kullanılmalı
    policy=model,
    ref_policy=ref_model,  # Referans politikayı ayrı bir nesne olarak geçiyoruz
    tokenizer=tokenizer,
    reward_model = trainer.reward_model
)